<a href="https://colab.research.google.com/github/Richiboy94/workplace/blob/main/rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors as mcolors
from scipy.stats import linregress
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans
from tabulate import tabulate
from collections import Counter

%matplotlib inline

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 1.2 | Loading the Dataset**

In [7]:
# Cargar el dataset
df = pd.read_csv("/content/drive/MyDrive/CAMBIX/RFM/RFM_Cambix_31.03.24.csv", sep=';')

**Paso 2.1 | Visión General del Conjunto de Datos**

In [10]:
# Filtrar los datos donde 'Recency' es menor o igual a 365 y mayor que 0
df_filtrado = df[(df['Recency'] <= 365.00) & (df['Recency'] > 0.00)]
df_x=df_filtrado

**Step 3.1 | Outlier Detection and Treatment**

In [11]:
# Inicializando el modelo IsolationForest con un parámetro de contaminación de 0.05
modelo = IsolationForest(contamination=0.05, random_state=0)

# Ajustando el modelo a nuestro conjunto de datos (convirtiendo el DataFrame a NumPy para evitar advertencias)
df_x['Puntuaciones_Valores_Atípicos'] = modelo.fit_predict(df_x.iloc[:, 1:].to_numpy())

# Creando una nueva columna para identificar valores atípicos (1 para valores normales y -1 para valores atípicos)
df_x['Es_Atípico'] = [1 if x == -1 else 0 for x in df_x['Puntuaciones_Valores_Atípicos']]


In [12]:
from sklearn.ensemble import IsolationForest

def remover_atípicos(df_x):
    # Configurar el modelo Isolation Forest
    clf = IsolationForest(contamination=0.05, random_state=42)

    # Ajustar el modelo y obtener las predicciones de valores atípicos (-1 para valores atípicos, 1 para valores normales)
    predicciones_atípicos = clf.fit_predict(df_x)

    # Filtrar solo los valores normales (1) y eliminar los outliers (-1)
    df_sin_atípicos = df_x[predicciones_atípicos == 1]

    return df_sin_atípicos


In [13]:
# Aplicar la función a tu conjunto de datos
df_y = remover_atípicos(df_x)

# **Segmentación RFM**

In [14]:
# Puntuación de recencia: 1 (muy malo) para aquellos con más días desde la última compra y 5 (muy bueno) para los que tienen menos días.
df_y["recency_score"] = pd.qcut(df_y['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Puntuación de frecuencia: 5 (muy bueno) para aquellos con alta frecuencia de compra y 1 (muy malo) para los que tienen menos frecuencia.
df_y["frequency_score"] = pd.qcut(df_y['Operaciones'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

# Puntuación monetaria: 5 (muy bueno) para aquellos con un alto valor de compra y 1 (muy malo) para los que tienen un valor de compra menor.
df_y["monetary_score"] = pd.qcut(df_y['MontoAcumulado'], 5, labels=[1, 2, 3, 4, 5])

# No incluiremos monetary_score.
df_y["RFM_SCORE"] = (df_y['recency_score'].astype(str)+df_y['frequency_score'].astype(str) + df_y['monetary_score'].astype(str))


In [15]:
seg_map = {
r'[1-2][1-4][1-2]': 'hibernating',
    r'[1-2][1-4][3-5]': 'at_Risk',
    r'[1-2][5][1-5]': 'cant_loose',
    r'[3-4][4-5][1-5]': 'Loyal',
        r'[3-4][1-3][1-5]': 'Needs Attention',
r'5[1-3][1-5]': 'promising',
    r'5[4-5][1-4]': 'potential_loyalists',
    r'5[4-5][5]': 'Champions'
}
# Crear la columna 'segment' en customer_data
df_y['segmento'] = df_y['RFM_SCORE'].replace(seg_map, regex=True)
if 'level_0' in df_y.columns:
    df_y.drop(columns=['level_0'], inplace=True)
df_y.reset_index(inplace=True)

,index,Num_Usuario,Operaciones,MontoAcumulado,Recency,Puntuaciones_Valores_Atípicos,Es_Atípico,recency_score,frequency_score,monetary_score,RFM_SCORE,segmento
0,26,11445,51,8244.00,4.324925,1,0,5,5,5,555,Champions
1,34,186,40,17148.18,64.495082,1,0,4,5,5,455,Loyal
2,36,111,38,15321.00,20.298330,1,0,5,5,5,555,Champions
3,38,225,37,5555.87,51.336042,1,0,5,5,5,555,Champions
4,39,1561,34,27136.14,32.226998,1,0,5,5,5,555,Champions


In [16]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Ruta donde guardar el archivo en tu Google Drive
output_path = '/content/drive/MyDrive/CAMBIX/RFM/RFM_Cambix_End.csv'

# Guarda el DataFrame como un archivo CSV
df_y.to_csv(output_path, index=False)

print(f"Dataset exportado exitosamente a: {output_path}")

Dataset exportado exitosamente a: /content/drive/MyDrive/CAMBIX/RFM/RFM_Cambix_End.csv
